----
<img src="../../../files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Content layer - Pricing stream - Used as a real-time data cache
This notebook demonstrates how to retrieve level 1 streaming data (such as trades and quotes) either directly from the Refinitiv Data Platform or via Refinitiv Workspace or CodeBook. The example shows how to define a Pricing stream object, which automatically manages a streaming cache available for access at any time. Your application can then reach into this cache and pull out real-time snapshots as Pandas DataFrames by just calling a simple access method.

Using a Pricing stream object that way prevents your application from sending too many requests to the platform. This is particularly useful if your application needs to retrieve real-time snapshots at regular and short intervals.

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [login](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) to the Refinitiv Developer Community portal you will get free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding the API usage, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help.

## Set the configuration file location
For a better ease of use, you have the option to set initialization parameters of the Refinitiv Data Library in the _refinitiv-data.config.json_ configuration file. This file must be located beside your notebook, in your user folder or in a folder defined by the _RD_LIB_CONFIG_PATH_ environment variable. The _RD_LIB_CONFIG_PATH_ environment variable is the option used by this series of examples. The following code sets this environment variable.      

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

## Some Imports to start with

In [2]:
import refinitiv.data as rd
from refinitiv.data.content import pricing
from pandas import DataFrame
from IPython.display import display, clear_output

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [3]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x7f7eec990b90 {name='workspace'}>

## Retrieve data

### Create and open a Pricing stream object

The Pricing stream object is created for a list of instruments and fields. The fields parameter is optionnal. If you omit it, the Pricing stream will retrieve all fields available for the requested instruments

In [4]:
stream = rd.content.pricing.Definition(
    universe = ['EUR=', 'GBP=', 'JPY=', 'CAD='], 
    fields = ['BID', 'ASK']
).get_stream()

The open method tells the Pricing stream object to subscribe to the streams of the requested instruments.

In [5]:
stream.open()

<OpenState.Opened: 1>

As soon as the open method returns, the stream object is ready to be used. Its internal cache is constantly kept updated with the latest streaming information received from Eikon / Refinitiv Workspace. All this happens behind the scene, waiting for your application to pull out data from the cache.  

### Extract snapshot data from the streaming cache
Once the stream is opened, you can use the get_snapshot method to pull out data from its internal cache. get_snapshot can be called any number of times. As these calls return the latest received values, successive calls to get_snapshot may return different values. Returned DataFrames do not change in real-time, get_snapshot must be called every time your application needs fresh values. 

In [6]:
df = stream.get_snapshot()
display(df)

,Instrument,BID,ASK
0,EUR=,1.133,1.1334
1,GBP=,1.3354,1.3355
2,JPY=,114.15,114.16
3,CAD=,1.2782,1.2783


### Get a snapshot for a subset of instruments and fields

In [7]:
df = stream.get_snapshot(
    universe = ['EUR=', 'GBP='], 
    fields = ['BID', 'ASK']
)
display(df)

,Instrument,BID,ASK
0,EUR=,1.1331,1.1335
1,GBP=,1.3354,1.3356


### Other options to get values from the streaming cache

#### Direct access to real-time fields

In [8]:
print('GBP/BID:', stream['GBP=']['BID'])
print('EUR/BID:', stream['EUR=']['BID'])

GBP/BID: 1.3353
EUR/BID: 1.1331


#### Direct acces to a streaming instrument

In [9]:
gbp = stream['GBP=']
print(gbp['BID'])

1.3353


#### Iterate on fields

In [10]:
print('GBP=')
for field_name, field_value in stream['GBP=']:
    print('\t' + field_name + ': ', field_value)
    
print('JPY=')
for field_name, field_value in stream['JPY=']:
    print('\t' + field_name + ': ', field_value)

GBP=
	BID:  1.3353
	ASK:  1.3357
JPY=
	BID:  114.14
	ASK:  114.16


#### Iterate on streaming instruments and fields

In [11]:
for streaming_instrument in stream:
    print(streaming_instrument.name)
    for field_name, field_value in streaming_instrument:
        print('\t' + field_name + ': ', field_value)

EUR=
	BID:  1.1331
	ASK:  1.1335
GBP=
	BID:  1.3353
	ASK:  1.3357
JPY=
	BID:  114.14
	ASK:  114.16
CAD=
	BID:  1.2777
	ASK:  1.2781


### Close the stream

In [12]:
stream.close()

<OpenState.Closed: 3>

Once closed is called the Pricing stream object stops updating its internal cache. The get_snapshot function can still be called but after the close it always return the same values. 

### Invalid or un-licensed instruments
What happens if you request using an invalid RIC or an instrument you are not entitled to?
Let's request a mixture of valid and invalid RICs

In [13]:
mixed = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD=', 'BADRIC'],
    fields=['BID', 'ASK']
).get_stream()

mixed.open()
mixed.get_snapshot()

,Instrument,BID,ASK
0,EUR=,1.1332,1.1333
1,GBP=,1.3353,1.3358
2,JPY=,114.14,114.17
3,CAD=,1.2781,1.2786
4,BADRIC,<NA>,<NA>


You can check the Status of any instrument, so lets check the invalid one

In [14]:
display(mixed['BADRIC'].status)

{'ID': 10,
 'Type': 'Status',
 'Key': {'Service': 'IDN_RDFNTS_CF', 'Name': 'BADRIC'},
 'State': {'Stream': 'Closed',
  'Data': 'Suspect',
  'Code': 'NotFound',
  'Text': 'The record could not be found'}}

As you will note, for an invalid instrument we get:   
{'status': <StreamState.Closed: 1>, **'code': 'NotFound'**, 'message': '** The Record could not be found'}   

However, if you are not licensed for the instrument you would see something like:  
{'status': <StreamState.Closed: 1>, **'code': 'NotEntitled'**, 'message': 'A21: DACS User Profile denied access to vendor'}   
**NOTE**: The exact wording of **message** can change over time - therefore,only use the **code** value for any programmatic decision making.

In [15]:
mixed.close()

<OpenState.Closed: 3>

## Close the session

In [16]:
rd.close_session()